In [1]:
import pandas as pd
import collections
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np

from google.colab import drive, files
drive.mount('/drive')

Mounted at /drive


In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV, ParameterGrid

In [5]:
#recipes = pd.read_csv('cleaned_recipes.csv', low_memory = False)

recipes = pd.read_csv('/drive/My Drive/Data 144/Recipes - Final Project/Pre-processed data/cleaned_recipes.csv', low_memory = False)

In [6]:
recipes.head()

,name,id,minutes,contributor_id,submitted,tags,nutrition,n_steps,steps,description,ingredients,n_ingredients,cuisine,calories (#),total fat (PDV),sugar (PDV),sodium (PDV),protein (PDV),saturated fat (PDV),carbohydrates (PDV)
0,arriba baked winter squash mexican style,137739,55,47892,2005-09-16,"['60-minutes-or-less', 'time-to-make', 'course...","['51.5', '0.0', '13.0', '0.0', '2.0', '0.0', '...",11,"['make a choice and proceed with recipe', 'dep...",autumn is my favorite time of year to cook! th...,"['winter squash', 'mexican seasoning', 'mixed ...",7,mexican,51.5,0.0,13.0,0.0,2.0,0.0,4.0
1,a bit different breakfast pizza,31490,30,26278,2002-06-17,"['30-minutes-or-less', 'time-to-make', 'course...","['173.4', '18.0', '0.0', '17.0', '22.0', '35.0...",9,"['preheat oven to 425 degrees f', 'press dough...",this recipe calls for the crust to be prebaked...,"['prepared pizza crust', 'sausage patty', 'che...",6,american,173.4,18.0,0.0,17.0,22.0,35.0,1.0
2,amish tomato ketchup for canning,44061,190,41706,2002-10-25,"['weeknight', 'time-to-make', 'course', 'main-...","['352.9', '1.0', '337.0', '23.0', '3.0', '0.0'...",5,['mix all ingredients& boil for 2 1 / 2 hours ...,my dh's amish mother raised him on this recipe...,"['tomato juice', 'apple cider vinegar', 'clove...",8,american,352.9,1.0,337.0,23.0,3.0,0.0,28.0
3,apple a day milk shake,5289,0,1533,1999-12-06,"['15-minutes-or-less', 'time-to-make', 'course...","['160.2', '10.0', '55.0', '3.0', '9.0', '20.0'...",4,"['combine ingredients in blender', 'cover and ...",NaN,"['vanilla ice cream', 'frozen apple juice conc...",4,american,160.2,10.0,55.0,3.0,9.0,20.0,7.0
4,aww marinated olives,25274,15,21730,2002-04-14,"['15-minutes-or-less', 'time-to-make', 'course...","['380.7', '53.0', '7.0', '24.0', '6.0', '24.0'...",4,['toast the fennel seeds and lightly crush the...,my italian mil was thoroughly impressed by my ...,"['fennel seeds', 'green olives', 'ripe olives'...",9,canadian,380.7,53.0,7.0,24.0,6.0,24.0,6.0


In [7]:
# fix ingrdients column format

recipes["ingredients"] = [string.strip("][").replace("'", '').split(', ') for string in recipes["ingredients"]]

In [8]:
# split X and y columns

X = recipes[['ingredients']]
y = recipes[['cuisine']]

In [9]:
# one-hot encode

from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

X = X.join(pd.DataFrame.sparse.from_spmatrix(
                mlb.fit_transform(X.pop('ingredients')),
                index=X.index,
                columns=mlb.classes_))

In [10]:
X.head()

,,"""adolphs meat tenderizer""","""annies goddess dressing""","""aunt janes krazy mixed up salt""","""bakers angel flake sweetened coconut""","""bakers germans chocolate""","""bakers joy""","""bakers semi-sweet chocolate""","""bakers yeast""","""betty crockers hamburger helper""",...,zatarians gumbo base mix,zest of,zesty hot sausage roll,zinfandel,ziploc bag,ziploc bags,ziti pasta,zucchini,zwieback toast,zwieback toast crumbs
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
# remove ingredients that were only used once

X = X.loc[:, (X.sum(axis=0) != 1)]
X.head()

,,"""aunt janes krazy mixed up salt""","""bakers germans chocolate""","""bakers semi-sweet chocolate""","""bakers yeast""","""brewers yeast""","""bushs red beans""","""campbells cheddar cheese soup""","""campbells condensed chicken broth""","""campbells condensed cream of celery soup""",...,yukon gold potato,yukon gold potatoes,zatarians gumbo base mix,zest of,zinfandel,ziploc bag,ziploc bags,ziti pasta,zucchini,zwieback toast
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3, random_state=42)

## Baseline

In [13]:
y_train.groupby(["cuisine"]).size().sort_values(ascending=False).head(3)

cuisine
american    21848
italian      4625
mexican      3786
dtype: int64

In [14]:
y_base_pred = ["american" for x in y_train["cuisine"]]
baseline_acc = accuracy_score(y_train, y_base_pred)
print(f'Baseline Train Accuracy: {baseline_acc:.4f}')

Baseline Train Accuracy: 0.4204


In [15]:
y_base_pred_test = ["american" for x in y_test["cuisine"]]
baseline_acc_test = accuracy_score(y_test, y_base_pred_test)
print(f'Baseline Test Accuracy: {baseline_acc_test:.4f}')

Baseline Test Accuracy: 0.4155


## Neural Network

In [ ]:
clf1 = MLPClassifier(hidden_layer_sizes=(10, ), activation='logistic', 
                     max_iter=100, solver='lbfgs', verbose=1,  random_state=42) 
clf1.fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:1109: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:549: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(activation='logistic', hidden_layer_sizes=(10,), max_iter=100,
              random_state=42, solver='lbfgs', verbose=1)

In [ ]:
print('Training accuracy:')
y_pred = clf1.predict(X_train)
print(accuracy_score(y_train,y_pred))

Training accuracy:
0.6499143786196679


In [ ]:
print('Test accuracy:')
y_pred_test = clf1.predict(X_test)
print(accuracy_score(y_test,y_pred_test))

Test accuracy:
0.5739169472502805


## Logistic Regression

In [16]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(random_state=42).fit(X_train, y_train)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:993: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [17]:
log_pred = clf.predict(X_train)
logreg_acc = accuracy_score(y_train, log_pred)
print(f'Logistic Regression Train Accuracy: {logreg_acc:.4f}')

Logistic Regression Train Accuracy: 0.7073


In [18]:
log_pred_test = clf.predict(X_test)
logreg_acc_test = accuracy_score(y_test, log_pred_test)
print(f'Logistic Regression Test Accuracy: {logreg_acc_test:.4f}')

Logistic Regression Test Accuracy: 0.5842


## Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)

<ipython-input-20-5db8bae6fca1>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf.fit(X_train, y_train)


RandomForestClassifier(random_state=42)

In [ ]:
y_pred_rf = rf.predict(X_train)
rf_acc = accuracy_score(y_train, y_pred_rf)
print(f'Random Forest Train Accuracy: {rf_acc:.4f}')

Random Forest Train Accuracy: 0.9912


In [ ]:
y_pred_rf_test = rf.predict(X_test)
rf_acc_test = accuracy_score(y_test, y_pred_rf_test)
print(f'Random Forest Test Accuracy: {rf_acc_test:.4f}')

Random Forest Test Accuracy: 0.5414


### CART

In [ ]:
from sklearn.tree import DecisionTreeClassifier

dtc = DecisionTreeClassifier(
    min_samples_leaf=5, 
    random_state = 42)

dtc = dtc.fit(X_train, y_train)

In [ ]:
y_pred_dc = dtc.predict(X_train)
dc_acc = accuracy_score(y_train, y_pred_dc)
print(f'CART Train Accuracy: {dc_acc:.4f}')

CART Train Accuracy: 0.5915


In [ ]:
y_pred_dc_test = dtc.predict(X_test)
dc_acc_test = accuracy_score(y_test, y_pred_dc_test)
print(f'CART Forest Test Accuracy: {dc_acc_test:.4f}')

CART Forest Test Accuracy: 0.5041


## Model Performance on Test Set

In [ ]:
dict = {"Model":["Baseline", "Neural Network", "Logistic Regression", "Random Forest", "CART"],
        "Accuracy": [baseline_acc_test,0.5739169472502805,logreg_acc_test,rf_acc_test,dc_acc_test],
}
model_performance = pd.DataFrame(data=dict)
model_performance.sort_values(["Accuracy"], ascending=False)

,Model,Accuracy
2,Logistic Regression,0.584153
1,Neural Network,0.573917
3,Random Forest,0.541414
4,CART,0.504108
0,Baseline,0.415533
